In [22]:
from quarto import objects, extensions
from quarto.players import MinMaxPlayer, RandomPlayer, ProPlayer
import logging
import math
from copy import deepcopy
from tqdm.notebook import tqdm
import numpy as np

In [23]:
def evaluate_minmax(matches = 1000, minmax_level=2, minmax_bound=math.factorial(6) ** 2, pro = False):
    scores = {
        'me': 0,
        'opponent': 0,
        'draw': 0
    }

    home = {
        0: 'me',
        1: 'opponent',
        -1: 'draw'
    }

    away = {
        0: 'opponent',
        1: 'me',
        -1: 'draw'
    }

    player_stats = np.array([0.0 for _ in range(6)])
    max_select = 0.0
    max_place = 0.0
    turns = 0.0

    for _ in tqdm(range(matches // 2), desc='Home games'):
        game = objects.Quarto()
        opponent = RandomPlayer(game)
        me = ProPlayer(game, bound=minmax_level, bound_value=minmax_bound) if pro else MinMaxPlayer(game, bound=minmax_level, bound_value=minmax_bound) 
        game.set_players((me, opponent))
        winner = home[game.run()]
        scores[winner] += 1
        player_stats += game.get_stats()[0]
        max_select = max(max_select, game.get_stats()[0][2])
        max_place = max(max_place, game.get_stats()[0][5])
        turns += 16 - (game.get_board_status() == -1).sum()

    for _ in tqdm(range(matches - matches // 2), desc='Away games'):
        game = objects.Quarto()
        opponent = RandomPlayer(game)
        me = ProPlayer(game, bound=minmax_level, bound_value=minmax_bound)  if pro else MinMaxPlayer(game, bound=minmax_level, bound_value=minmax_bound) 
        game.set_players((opponent, me))
        winner = away[game.run()]  
        scores[winner] += 1
        player_stats += game.get_stats()[1]
        max_select = max(max_select, game.get_stats()[1][2])
        max_place = max(max_place, game.get_stats()[1][5])

    player_stats[2] = max_select
    player_stats[5] = max_place
    logging.warning(f"Stats: {scores}") 
    logging.warning(f"Average turns: {2 * turns / matches }") 
    logging.warning(f'Player 0: Avg select {player_stats[0]/player_stats[1]} s (max {player_stats[2]} s), Avg place {player_stats[3]/player_stats[4]} s (max {player_stats[5]} s)')

    return scores

In [21]:
def evaluate_minmaxes(matches = 1000, minmax_levels=(3,2), minmax_bounds=(math.factorial(5) ** 2,math.factorial(6) ** 2), pros = (True, True)):
    scores = {
        'me': 0,
        'opponent': 0,
        'draw': 0
    }

    home = {
        0: 'me',
        1: 'opponent',
        -1: 'draw'
    }

    away = {
        0: 'opponent',
        1: 'me',
        -1: 'draw'
    }

    player_stats = np.array([0.0 for _ in range(6)])
    max_select = 0.0
    max_place = 0.0
    turns = 0.0

    for _ in tqdm(range(matches // 2), desc='Home games'):
        game = objects.Quarto()
        opponent = ProPlayer(game, bound=minmax_levels[1], bound_value=minmax_bounds[1]) if pros[1] else MinMaxPlayer(game, bound=minmax_levels[1], bound_value=minmax_bounds[1])
        me = ProPlayer(game, bound=minmax_levels[0], bound_value=minmax_bounds[0]) if pros[0] else MinMaxPlayer(game, bound=minmax_levels[0], bound_value=minmax_bounds[0])
        game.set_players((me, opponent))
        winner = home[game.run()]
        scores[winner] += 1
        player_stats += game.get_stats()[0]
        max_select = max(max_select, game.get_stats()[0][2])
        max_place = max(max_place, game.get_stats()[0][5])
        turns += 16 - (game.get_board_status() == -1).sum()

    for _ in tqdm(range(matches - matches // 2), desc='Away games'):
        game = objects.Quarto()
        opponent = ProPlayer(game, bound=minmax_levels[1], bound_value=minmax_bounds[1]) if pros[1] else MinMaxPlayer(game, bound=minmax_levels[1], bound_value=minmax_bounds[1])
        me = ProPlayer(game, bound=minmax_levels[0], bound_value=minmax_bounds[0]) if pros[0] else MinMaxPlayer(game, bound=minmax_levels[0], bound_value=minmax_bounds[0])
        game.set_players((opponent, me))
        winner = away[game.run()]  
        scores[winner] += 1
        player_stats += game.get_stats()[1]
        max_select = max(max_select, game.get_stats()[1][2])
        max_place = max(max_place, game.get_stats()[1][5])

    player_stats[2] = max_select
    player_stats[5] = max_place
    logging.warning(f"Stats: {scores}") 
    logging.warning(f"Average turns: {2 * turns / matches}") 
    logging.warning(f'Player 0: Avg select {player_stats[0]/player_stats[1]} s (max {player_stats[2]} s), Avg place {player_stats[3]/player_stats[4]} s (max {player_stats[5]} s)')

    return scores

In [26]:
evaluate_minmax(1000, 1, 1, True)
evaluate_minmax(1000, 2, math.factorial(6)**2, True)
evaluate_minmax(1000, 3, math.factorial(5)**2, True)

Home games:   0%|          | 0/500 [00:00<?, ?it/s]

Away games:   0%|          | 0/500 [00:00<?, ?it/s]

Home games:   0%|          | 0/500 [00:00<?, ?it/s]

Away games:   0%|          | 0/500 [00:00<?, ?it/s]

Home games:   0%|          | 0/500 [00:00<?, ?it/s]

Away games:   0%|          | 0/500 [00:00<?, ?it/s]

{'me': 993, 'opponent': 4, 'draw': 3}

In [27]:
evaluate_minmaxes(100, (3,1), (math.factorial(5)**2,1), (True, True))
evaluate_minmaxes(100, (3,2), (math.factorial(5)**2,math.factorial(6)**2), (True, True))
evaluate_minmaxes(100, (3,3), (math.factorial(5)**2,math.factorial(5)**2), (True, False))

Home games:   0%|          | 0/50 [00:00<?, ?it/s]

Away games:   0%|          | 0/50 [00:00<?, ?it/s]

Home games:   0%|          | 0/50 [00:00<?, ?it/s]

Away games:   0%|          | 0/50 [00:00<?, ?it/s]

Home games:   0%|          | 0/50 [00:00<?, ?it/s]

Away games:   0%|          | 0/50 [00:00<?, ?it/s]

{'me': 39, 'opponent': 15, 'draw': 46}

In [28]:
evaluate_minmaxes(20, (3,3), (math.factorial(5)**2,math.factorial(6)**2), (True, False))
evaluate_minmaxes(20, (3,3), (math.factorial(5)**2,math.factorial(6)**2), (True, True))
evaluate_minmaxes(20, (3,3), (math.factorial(6)**2,math.factorial(6)**2), (True, False))

Home games:   0%|          | 0/10 [00:00<?, ?it/s]

Away games:   0%|          | 0/10 [00:00<?, ?it/s]

Home games:   0%|          | 0/10 [00:00<?, ?it/s]

Away games:   0%|          | 0/10 [00:00<?, ?it/s]

Home games:   0%|          | 0/10 [00:00<?, ?it/s]

Away games:   0%|          | 0/10 [00:00<?, ?it/s]

{'me': 16, 'opponent': 2, 'draw': 2}

In [5]:
game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 3, math.factorial(5)**2), MinMaxPlayer(game, 2, math.factorial(6)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 2, math.factorial(6)**2), MinMaxPlayer(game, 3, math.factorial(5)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

Winner is 0
Played 8 turns
Player 0: Avg select 0.02750682830810547 s (max 0.11002731323242188 s), Avg place 0.8444941639900208 s (max 2.488685131072998 s)
Player 1: Avg select 0.0 s (max 0.0 s), Avg place 0.06976926326751709 s (max 0.10202717781066895 s)
Winner is -1
Played 16 turns
Player 0: Avg select 0.01350286602973938 s (max 0.10802292823791504 s), Avg place 0.20910561084747314 s (max 1.3992741107940674 s)
Player 1: Avg select 0.0 s (max 0.0 s), Avg place 0.5132704377174377 s (max 1.6715977191925049 s)


In [7]:
game = objects.Quarto()
game.set_players((ProPlayer(game, 3, math.factorial(5)**2), MinMaxPlayer(game, 2, math.factorial(6)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 2, math.factorial(6)**2), ProPlayer(game, 3, math.factorial(5)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

Winner is 0
Played 8 turns
Player 0: Avg select 0.0 s (max 0.0 s), Avg place 0.8806970119476318 s (max 2.5685813426971436 s)
Player 1: Avg select 0.0 s (max 0.0 s), Avg place 0.07252073287963867 s (max 0.10703182220458984 s)
Winner is 1
Played 11 turns
Player 0: Avg select 0.017670631408691406 s (max 0.10602378845214844 s), Avg place 0.05403909683227539 s (max 0.09416604042053223 s)
Player 1: Avg select 0.0196044921875 s (max 0.0980224609375 s), Avg place 0.3853995402654012 s (max 1.178267240524292 s)


In [9]:
game = objects.Quarto()
game.set_players((ProPlayer(game, 3, math.factorial(5)**2), MinMaxPlayer(game, 3, math.factorial(5)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

game = objects.Quarto()
game.set_players((MinMaxPlayer(game, 3, math.factorial(5)**2), ProPlayer(game, 3, math.factorial(5)**2)))
winner = game.run()
print(f'Winner is {winner}')
print(f'Played {sum(sum(game.get_board_status() != -1))} turns')
_ = game.get_stats(True)

Winner is 0
Played 14 turns
Player 0: Avg select 0.0 s (max 0.0 s), Avg place 0.5296737125941685 s (max 1.6153655052185059 s)
Player 1: Avg select 0.00014284678867885044 s (max 0.0009999275207519531 s), Avg place 0.4162571770804269 s (max 1.4633314609527588 s)
Winner is 1
Played 13 turns
Player 0: Avg select 0.015146153313773019 s (max 0.10602307319641113 s), Avg place 0.6171410878499349 s (max 1.5023398399353027 s)
Player 1: Avg select 0.015670418739318848 s (max 0.09402251243591309 s), Avg place 0.41480677468436106 s (max 1.39430570602417 s)
